In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,LocallyConnected2D,BatchNormalization,AveragePooling2D
from tensorflow.keras.preprocessing import image

In [2]:
nRowsRead = 20000 # specify 'None' if want to read whole file
# list_bbox_celeba.csv has 202599 rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('list_bbox_celeba.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'list_bbox_celeba.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 20000 rows and 5 columns


In [3]:
df.columns

Index(['image_id', 'x_1', 'y_1', 'width', 'height'], dtype='object')

In [4]:
images_data_path = "C:/Users/foreh/Documents/celebaRepo/FacialAttrCNN/celeba/greyscale"

In [5]:
width = 178
height = 218
image_size = (width,height)
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)

In [6]:
images = images_data # list of array of images
labels = df[["image_id", "width", "height"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(6))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(5))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

Samples Training Set: 7000
Samples in Validation Data: 1500
Samples in Test Data: 1500


In [10]:
model = Sequential()
model.add(tf.keras.Input(shape=(image_size[1],image_size[0], 1)))
model.add(Conv2D(24, (3,3), strides=(3,3),padding="valid"))
model.add(Conv2D(8, (3,3), strides=(2,1)))
model.add(MaxPooling2D(2))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Conv2D(8, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(44))
model.add(Dense(32))
model.add(Dense(2, activation="linear"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 72, 59, 24)        240       
                                                                 
 conv2d_6 (Conv2D)           (None, 35, 57, 8)         1736      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 28, 8)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 17, 28, 8)         0         
                                                                 
 batch_normalization_2 (Batc  (None, 17, 28, 8)        32        
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 15, 26, 16)       

In [11]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer = opt,loss="huber_loss",metrics = ['acc', 'mse'])

In [12]:
training_process = model.fit(X_train,y_train,epochs = 10,validation_data = (X_val,y_val),batch_size=8,shuffle = True)

Epoch 1/10
875/875 [==============================] - 14s 15ms/step - loss: 108.6823 - acc: 0.9236 - mse: 38482.0234 - val_loss: 142.7216 - val_acc: 0.9993 - val_mse: 32350.8770
Epoch 2/10
875/875 [==============================] - 13s 15ms/step - loss: 88.3402 - acc: 0.9993 - mse: 29567.3301 - val_loss: 82.5533 - val_acc: 0.9993 - val_mse: 23023.1816
Epoch 3/10
875/875 [==============================] - 13s 15ms/step - loss: 82.3237 - acc: 0.9993 - mse: 27141.1113 - val_loss: 77.0700 - val_acc: 0.9993 - val_mse: 19669.1348
Epoch 4/10
875/875 [==============================] - 13s 15ms/step - loss: 79.1303 - acc: 0.9993 - mse: 26038.6680 - val_loss: 74.7627 - val_acc: 0.9993 - val_mse: 19682.3672
Epoch 5/10
875/875 [==============================] - 13s 15ms/step - loss: 78.0553 - acc: 0.9993 - mse: 25997.7188 - val_loss: 94.0619 - val_acc: 0.9993 - val_mse: 25523.9609
Epoch 6/10
875/875 [==============================] - 13s 15ms/step - loss: 76.8000 - acc: 0.9993 - mse: 25440.5312 - 

KeyboardInterrupt: 

In [17]:
def image_from_index(index, size=(image_size), path=images_data_path):
        path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
        # to read the image
        image = PIL.Image.open(path).resize(size)
        image_array = np.asarray(image)

In [22]:
#model.save("faceSizeRegression/iter2") #iter1: .9993 accuracy
model3 = tf.keras.models.load_model("faceSizeRegression/iter1")
#model3.evaluate(X_val,y_val)
img = image_from_index(999)
tf.keras.training.predict(model3, img)

AttributeError: module 'tensorflow.keras' has no attribute 'training'

In [6]:
images = images_data # list of array of images
labels = df[["image_id", "x_1"]]#, "y_1"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(6))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(5))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

Samples Training Set: 14000
Samples in Validation Data: 3000
Samples in Test Data: 3000


In [7]:
model2 = Sequential()
model2.add(tf.keras.Input(shape=(image_size[1],image_size[0], 1)))
model2.add(AveragePooling2D((3,3)))
model2.add(Conv2D(128, (3,3)))
model2.add(Conv2D(64, (3,3)))
model2.add(Dropout(.2))
model2.add(BatchNormalization())
model2.add(Conv2D(64, (3,3), strides=(3,3)))
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Dense(32))
model2.add(Dense(1, activation="linear"))
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 average_pooling2d (AverageP  (None, 72, 59, 1)        0         
 ooling2D)                                                       
                                                                 
 conv2d (Conv2D)             (None, 70, 57, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 68, 55, 64)        73792     
                                                                 
 dropout (Dropout)           (None, 68, 55, 64)        0         
                                                                 
 batch_normalization (BatchN  (None, 68, 55, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 18, 64)        3

In [8]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)

model2.compile(optimizer = "adam",loss="huber_loss",metrics = ['acc', 'mse'])

In [9]:
training_process = model2.fit(X_train,y_train,epochs = 20,validation_data = (X_val,y_val),batch_size=8,shuffle = True)

Epoch 1/20
 836/1750 [=============>................] - ETA: 27s - loss: 95.2925 - acc: 0.0076 - mse: 29064.0312

KeyboardInterrupt: 

In [11]:
model2.evaluate(X_val, y_val)

47/47 [==============================] - 2s 33ms/step - loss: 64.2126 - acc: 0.7313 - mse: 15345.2852


[64.21255493164062, 0.731333315372467, 15345.28515625]

In [17]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
  
train_generator = train_datagen.flow_from_dataframe(df, x_col="image_id", y_col="x_1", 
                                                      batch_size = 20)
  
validation_generator = test_datagen.flow_from_directory(images_data_path, target_size =(178, 218),
                                                      batch_size = 20, class_mode = "binary")

TypeError: If class_mode="categorical", y_col="x_1" column values must be type string, list or tuple.

history = model2.fit_generator(train_generator, steps_per_epoch = 100, epochs = 30,
validation_data = validation_generator, validation_steps = 50)